In [1]:
import joblib

In [2]:
complete_features_matrix = joblib.load('algebra_2005_2008_pickles/temporal_features_matrix.compressed')
complete_features_matrix = complete_features_matrix.tocsr()
complete_features_matrix

/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2652: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


<8483920x217679 sparse matrix of type '<type 'numpy.float64'>'
	with 136587838 stored elements in Compressed Sparse Row format>

In [3]:
labels = joblib.load('algebra_2005_2008_pickles/labels.compressed')

## Logreg with default with standard scaler

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
logreg = LogisticRegression(n_jobs=-1, verbose=2)
scaler = StandardScaler(with_mean=False, copy=False)

In [6]:
clf = Pipeline([('scaler', scaler), ('logreg', logreg)])

In [7]:
clf.fit(complete_features_matrix, labels)

[LibLinear]

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:924: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('scaler', StandardScaler(copy=False, with_mean=False, with_std=True)), ('logreg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=2, warm_start=False))])

In [8]:
joblib.dump(clf, 'algebra_2005_2008_pickles/LogRegdefault_temporal_features.compressed', compress=3)

['algebra_2005_2008_pickles/LogRegdefault_temporal_features.compressed']

## Logreg with C10 with standard scaler

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [5]:
logreg = LogisticRegression(C=10, n_jobs=-1, verbose=2)
scaler = StandardScaler(with_mean=False, copy=False)

In [6]:
clf = Pipeline([('scaler', scaler), ('logreg', logreg)])

In [7]:
clf.fit(complete_features_matrix, labels)

[LibLinear]

/usr/local/lib/python2.7/dist-packages/sklearn/svm/base.py:924: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('scaler', StandardScaler(copy=False, with_mean=False, with_std=True)), ('logreg', LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=2, warm_start=False))])

In [8]:
joblib.dump(clf, 'algebra_2005_2008_pickles/LogRegC10_temporal_features.compressed', compress=3)

['algebra_2005_2008_pickles/LogRegC10_temporal_features.compressed']

## Testing

In [9]:
test = joblib.load('algebra_2005_2008_pickles/temporal_test_features_matrix')

In [10]:
probas = clf.predict_proba(test)
probas

array([[ 0.15085894,  0.84914106],
       [ 0.13998024,  0.86001976],
       [ 0.0126116 ,  0.9873884 ],
       ..., 
       [ 0.23975005,  0.76024995],
       [ 0.05324343,  0.94675657],
       [ 0.03447232,  0.96552768]])

In [11]:
ones = []
zeros = []
for zero_prob, one_prob in probas:
    if one_prob > zero_prob:
        ones.append(one_prob)
    else:
        zeros.append(one_prob)
        
print len(ones), len(zeros)

485231 23681


In [12]:
import csv
with open('algebra_2008_2009_submissiona.txt', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='\t')
    spamwriter.writerow(['Row', 'Correct First Attempt'])
    for index, (zero_prob, one_prob) in enumerate(probas):
        spamwriter.writerow([index+1, float(one_prob)])